In [1]:
import requests
from google.cloud import bigquery
import pandas as pd
from datetime import datetime
from google.oauth2 import service_account
url = 'https://api.binance.com/api/v3/historicalTrades'

r = requests.get(url, params=dict(symbol ="BTCUSDT"))

In [2]:
import pandas as pd
data = r.json()
df = pd.DataFrame(data)
df

,id,price,qty,quoteQty,time,isBuyerMaker,isBestMatch
0,3478762553,72097.40000000,0.10000000,7209.74000000,1710306472204,True,True
1,3478762554,72097.41000000,0.00208000,149.96261280,1710306472221,False,True
2,3478762555,72097.40000000,0.02200000,1586.14280000,1710306472969,True,True
3,3478762556,72097.41000000,0.00105000,75.70228050,1710306472988,False,True
4,3478762557,72097.41000000,0.00105000,75.70228050,1710306472988,False,True
...,...,...,...,...,...,...,...
495,3478763048,72121.36000000,0.00027000,19.47276720,1710306514758,False,True
496,3478763049,72121.36000000,0.00670000,483.21311200,1710306514834,False,True
497,3478763050,72121.35000000,0.02518000,1816.01559300,1710306516018,True,True
498,3478763051,72121.36000000,0.00207000,149.29121520,1710306516575,False,True


In [3]:
trade_lst = ["BTCUSDT", "AAVEUSDT", "STXUSDT", "ARBUSDT"]
project_id = 'binance-416514'
dwh = 'dwh'
scd = 'scd'
keypath ='/home/tin/credentials/binance-cred.json'

credentials = service_account.Credentials.from_service_account_file(
            keypath, 
            scopes= ["https://www.googleapis.com/auth/cloud-platform"],
            )
# credentials= keypath ,scopes= ["https://www.googleapis.com/auth/cloud-platform"]
client = bigquery.Client(credentials=credentials)

def extract_data(trade):
    table_id = f'{project_id}.{dwh}.{trade}'

    print(f'now we get {table_id}')
    r = requests.get(url, params=dict(symbol = trade))
    print('getting json')
    data = r.json()
    print('getting dataframe')
    df = pd.DataFrame(data
                    # ,index = [0]
            )
    df['snapshot_date']= datetime.now()

    print(f'start uploading to {table_id}')

    job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_TRUNCATE",
    )
    job = client.load_table_from_dataframe(
        df, 
        table_id, 
        job_config=job_config
    )  # Make an API request.
    job.result()  # Wait for the job to complete.
    client.get_table(table_id)

In [4]:
for i in trade_lst:
    extract_data(i)

now we get binance-416514.dwh.BTCUSDT
getting json
getting dataframe
start uploading to binance-416514.dwh.BTCUSDT
now we get binance-416514.dwh.AAVEUSDT
getting json
getting dataframe
start uploading to binance-416514.dwh.AAVEUSDT
now we get binance-416514.dwh.STXUSDT
getting json
getting dataframe
start uploading to binance-416514.dwh.STXUSDT
now we get binance-416514.dwh.ARBUSDT
getting json
getting dataframe
start uploading to binance-416514.dwh.ARBUSDT
